In [1]:
import pandas as pd
from pprint import pprint
from datetime import datetime

In [2]:
sales_order_df = pd.read_parquet(
    "./test_data/sales_order_2025-6-9_13-11-9_744000.parquet"
)
print(sales_order_df.head(2))


   sales_order_id              created_at            last_updated  design_id  \
0               2 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186          3   
1               3 2022-11-03 14:20:52.188 2022-11-03 14:20:52.188          4   

   staff_id  counterparty_id  units_sold unit_price  currency_id  \
0        19                8       42972       3.94            2   
1        10                4       65839       2.91            3   

  agreed_delivery_date agreed_payment_date  agreed_delivery_location_id  
0           2022-11-07          2022-11-08                            8  
1           2022-11-06          2022-11-07                           19  


In [ ]:
def dim_date_dataframe(start_date: str, end_date: str):
    all_dates = pd.date_range(start=start_date, end=end_date,,tz="Europe/London", normalize=True)
    date_dataframe = pd.DataFrame({"date_value": all_dates})

    # Create key: YYYYMMDD as INT

    date_dataframe["year"] = date_dataframe["date_value"].dt.year
    date_dataframe["month"] = date_dataframe["date_value"].dt.month
    date_dataframe["day"] = date_dataframe["date_value"].dt.day
    date_dataframe["day_of_week"] = date_dataframe["date_value"].dt.weekday + 1
    date_dataframe["day_name"] = date_dataframe["date_value"].dt.day_name()
    date_dataframe["month_name"] = date_dataframe["date_value"].dt.month_name()
    date_dataframe["quarter"] = date_dataframe["date_value"].dt.quarter
    date_dataframe["date_id"] = date_dataframe["date_value"].dt.date

    date_dataframe = date_dataframe[
        [
            "date_id",
            "year",
            "month",
            "day",
            "day_of_week",
            "day_name",
            "month_name",
            "quarter",
        ]
    ]

    return date_dataframe

In [4]:
dim_date = dim_date_dataframe("20221102", "20500101")
pprint(dim_date.head(15))

       date_id  year  month  day  day_of_week   day_name month_name  quarter
0   2022-11-02  2022     11    2            3  Wednesday   November        4
1   2022-11-03  2022     11    3            4   Thursday   November        4
2   2022-11-04  2022     11    4            5     Friday   November        4
3   2022-11-05  2022     11    5            6   Saturday   November        4
4   2022-11-06  2022     11    6            7     Sunday   November        4
5   2022-11-07  2022     11    7            1     Monday   November        4
6   2022-11-08  2022     11    8            2    Tuesday   November        4
7   2022-11-09  2022     11    9            3  Wednesday   November        4
8   2022-11-10  2022     11   10            4   Thursday   November        4
9   2022-11-11  2022     11   11            5     Friday   November        4
10  2022-11-12  2022     11   12            6   Saturday   November        4
11  2022-11-13  2022     11   13            7     Sunday   November        4

In [ ]:
def create_fact_sales_order_from_df(data_frame: pd.DataFrame) -> pd.DataFrame:
    fact_sales_order_df = data_frame.copy()
    # Create columns
    # fact_sales_order_df["sales_record_id"] = range(1, len(fact_sales_order_df) + 1)

    fact_sales_order_df["created_date"] = pd.to_datetime(
        fact_sales_order_df["created_at"]
    ).dt.date
    fact_sales_order_df["last_updated_date"] = pd.to_datetime(
        fact_sales_order_df["last_updated"]
    ).dt.date
    fact_sales_order_df["agreed_payment_date"] = pd.to_datetime(
        fact_sales_order_df["agreed_payment_date"]
    ).dt.date
    fact_sales_order_df["agreed_delivery_date"] = pd.to_datetime(
        fact_sales_order_df["agreed_delivery_date"]
    ).dt.date

    fact_sales_order_df["created_time"] = pd.to_datetime(
        fact_sales_order_df["created_at"]
    ).dt.time
    fact_sales_order_df["last_updated_time"] = pd.to_datetime(
        fact_sales_order_df["last_updated"]
    ).dt.time

    fact_sales_order_df.drop(["created_at", "last_updated"], axis=1, inplace=True)

    return fact_sales_order_df


new_df = create_fact_sales_order_from_df(sales_order_df)
# pprint(fact_sales_order_df.head(1)['created_date'])
# pprint(fact_sales_order_df.columns.tolist())

pprint(new_df)

       sales_order_id  design_id  staff_id  counterparty_id  units_sold  \
0                   2          3        19                8       42972   
1                   3          4        10                4       65839   
2                   4          4        10               16       32069   
3                   5          7        18                4       49659   
4                   6          3        13               18       83908   
...               ...        ...       ...              ...         ...   
14587           14588         91        18                8       11213   
14588           14589        486        11               16       92009   
14589           14590        282         4               16       54573   
14590           14591        421         7               18       41622   
14591           14592        545        12               17       83748   

      unit_price  currency_id agreed_delivery_date agreed_payment_date  \
0           3.94         

In [6]:
# fact_sales_order_df = fact_sales_order_df.merge(
#     dim_date[["date_id"]],
#     how="left",
#     left_on="created_date",
#     right_on="date_id"
# ).rename(columns={"date_id": "created_date"})

# pprint(sales_order_df.head(1))

